In [11]:
import datetime
import requests
import pandas as pd
import hopsworks
import datetime
import matplotlib.pyplot as plt
import json
import re
import os
import sys
from dotenv import load_dotenv
import warnings
sys.path.append(os.path.abspath(os.path.join('..', 'functions')))
import util

warnings.filterwarnings("ignore")

In [12]:
load_dotenv()

HOPSWORK_API_KEY = os.getenv("HOPSWORK_API_KEY")
proj = hopsworks.login()
util.purge_project(proj)

Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated
Connected. Call `.close()` to terminate connection gracefully.


RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/variables/docker_base_image_python_version). Server response: 
HTTP code: 401, HTTP reason: Unauthorized, body: b'{"errorCode":200003,"errorMsg":"Invalidated Api key."}', error code: 200003, error msg: Invalidated Api key., user msg: 